# Import Libraries

In [1]:
!pip install texthero

     |████████████████████████████████| 245kB 6.3MB/s 
     |████████████████████████████████| 1.5MB 19.0MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import texthero as hero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



# Read Dataset

In [6]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [7]:
# products against which complaints have been filed
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

In [8]:
# companies against which complaints have been filed
df['Company'].value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Rapid Auto Loans LLC                     1   
Kashia Services                          1   
Hodosh, Lyon & Hammer, PC                1   
BERKSHIRE HILLS BANCORP, INC             1   
National Credit Processing Inc.          1   
Name: Company, Length: 2197, dtype: int64

In [9]:
df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

# Pre-processing

In [10]:
df['clean_complaints'] = hero.clean(df['complaints'])

In [11]:
df.head(10)

,complaints,Product,Company,clean_complaints
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION,auto loan opened xx xx xxxx nc bb name never north carolina never resident filed dispute twice credit bureaus times bb claimed accurate loan aware today tried contact bb multiple times never gotten live person n drive never owned car knowledge account checked xxxxxxxx xxxx noticed tried twice dispute additionally never received bills information account last resort trying remove fraudulent loan account
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings,xxxx noticed debt credit believed mine thought speedy cash bought one old debts sold xxxx xxxx xxxx xxxx contacted xxxx xxxx xxxx xxxx several attempts giving full name nothing came system gave social rep said account popped tell account someone elses name continued let make payment payment confirmation number xxxx realizing account called back get money back inform mistake told needed mail ftc report dispute letter get money back completed called said transferred account back speedy cash fraud review would need contact contacting told get money back issue im representative xxxx xxxx played blind obvious fraud let innocent person make payment someone elses debt want money back
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION,stated capital one xxxx xx xx xxxx xxxx wife went several car dealerships request car loan get used car however according credit requirements unfortunately credit score insufficient car loan approval time seemed though pulled credit report multiple times
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has n

# Train-test split

In [12]:
X_train, X_hold = train_test_split(df, test_size=0.3, random_state=111)

In [13]:
X_train['Product'].value_counts()

Debt collection                15281
Credit card or prepaid card    9263 
Mortgage                       6815 
Checking or savings account    4864 
Student loan                   2078 
Vehicle loan or lease          1916 
Name: Product, dtype: int64

# Feature Extraction

In [14]:
lemmatizer = WordNetLemmatizer()

def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and 'xx' not in word)] 
   tokens = [lemmatizer.lemmatize(item) for item in tokens]
   return tokens

In [15]:
# TfidfVectorizer will act as CountVectorizer when use_idf=False, norm=None
# Include a word only if it is present in minimum 50 documents 
# Include a word only if it is present in maximum 75% of the documents. If a word is present in >75% of documents, we are considering it as a common word. 
vectorizer_tf = TfidfVectorizer(tokenizer = tokenize, 
                                stop_words = None , 
                                max_df = 0.75, 
                                min_df = 50, 
                                max_features = 10000, 
                                use_idf = False, 
                                norm = None)

tf_vectors = vectorizer_tf.fit_transform(X_train['clean_complaints']) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.



In [16]:
tf_vectors.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Training

In [17]:
# Here we will use Latent Dirichlet Allocation (LDA)
# We can also use other techniques like Non Negative Matrix Factorization (NMF) - decomposition.NMF
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=10, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)

In [18]:
# We factorize/decompose a matrix X into two matrices W and H so that X = WH

# W matrix tells which topic is most prominent in each document
# We can see that document 1 is classified as Topic 5 because of highest probability of 0.8.
W1 = lda.fit_transform(tf_vectors) 
W1

array([[5.23391868e-03, 1.74945305e-01, 5.24257954e-03, 5.23284325e-03,
        8.04115083e-01, 5.23027049e-03],
       [6.29333467e-01, 1.42136098e-02, 1.03206633e-01, 6.59693485e-04,
        2.40722080e-01, 1.18645167e-02],
       [1.19791030e-02, 9.40213989e-01, 1.19792084e-02, 1.19157189e-02,
        1.19545036e-02, 1.19574767e-02],
       ...,
       [8.48875244e-01, 3.01510909e-03, 3.00637765e-03, 2.99042545e-03,
        1.39092805e-01, 3.02003870e-03],
       [6.30461250e-01, 1.69234808e-01, 1.67807457e-02, 1.67042877e-02,
        1.67196763e-02, 1.50099232e-01],
       [1.40555204e-03, 1.41590572e-03, 4.87089496e-01, 5.07261659e-01,
        1.41121341e-03, 1.41617315e-03]])

In [19]:
# H matrix tells which word is most prominent in each topic
H1 = lda.components_
H1

In [20]:
# Select only Top 15 words in each topic,
# so that we can know what the customer is talking about in each topic
num_words = 15

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [21]:
# Let's see what the topics are talking about
topics

['account card bank credit charge told money check time called chase customer transaction balance closed',
 'debt credit account report collection company information reporting agency bureau letter creditor theft identity sent',
 'payment loan mortgage month told time paid company called late said insurance received account year',
 'loan mortgage state court property document attorney agreement contract sale modification consumer foreclosure complaint usaa',
 'received number letter information phone email sent called address asked contact account time told service',
 'loan credit payment fargo score year application rate student income time program forbearance navient day']

In [22]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [23]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.01,0.17,0.01,0.01,0.80,0.01,4
Doc1,0.63,0.01,0.10,0.00,0.24,0.01,0
Doc2,0.01,0.94,0.01,0.01,0.01,0.01,1
Doc3,0.61,0.00,0.00,0.00,0.11,0.26,0
Doc4,0.91,0.01,0.01,0.06,0.01,0.01,0
...,...,...,...,...,...,...,...
Doc40212,0.88,0.00,0.00,0.00,0.11,0.00,0
Doc40213,0.53,0.22,0.18,0.00,0.06,0.00,0
Doc40214,0.85,0.00,0.00,0.00,0.14,0.00,0
Doc40215,0.63,0.17,0.02,0.02,0.02,0.15,0


In [24]:
X_train.head()

complaints  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   clean_complaints
30936  Throughout XX/XX/2019, I have received multiple calls from this company. I have also asked them to not call me or anyone associated with me. They have called my mother in law way too many times asking for my partner. They have called my partner multiple times asking for me. I'm not sure what kind of operation is being ran there, but it needs to stop. I'm giving notice to cease all contact with me or anyone else except the creditor about this claimed debt. Only contact me in writing, not by phone. Thank you.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# Testing

In [32]:
WHold = lda.transform(vectorizer_tf.transform(X_hold['clean_complaints']))

In [33]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [34]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.98,0.00,0.02,0.00,0.00,0.00,0
Doc1,0.00,0.00,0.46,0.12,0.41,0.00,2
Doc2,0.78,0.01,0.19,0.01,0.01,0.01,0
Doc3,0.00,0.00,0.36,0.00,0.14,0.50,5
Doc4,0.44,0.00,0.35,0.00,0.00,0.20,0
...,...,...,...,...,...,...,...
Doc17231,0.00,0.00,0.28,0.00,0.51,0.20,4
Doc17232,0.70,0.00,0.00,0.05,0.24,0.00,0
Doc17233,0.33,0.00,0.51,0.04,0.00,0.13,2
Doc17234,0.42,0.08,0.12,0.00,0.38,0.00,0


In [35]:
X_hold.head()

,complaints,Product,Company,clean_complaints
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION,business checking account bb xx xx attempted deposit check account received message stating monthly mobile deposit limit confused first month deposited checks since previous month called bb said couldnt deposit checks business accounts via mobile app even though done instructed open personal account could deposit checks via mobile app told opened account online would immediate access could link personal business accounts immediately able transfer money xx xx xxxx opened personal account online though successfully opened online online access promised traveling area bb branches could go branch xx xx intervening time business account became overdrawn money personal account bring back black bb could make transfer went branch time incurred astonishing overdraft fees business account way transferring money two accounts went branch met xxxx xxxx xxxx xxxx county florida investigated accounts told would link would online access xxxx hours online access still never material able transfer money received additional overdraft fee brought total overdraft fees opened second account bb ts advice could customer send bank wire directly business account first week xxxx keep account positive total deposits two accounts always positive requested refund told would refund fees
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you nee